In [1]:
import pandas as pd
import requests
import re
import numpy as np
import ast

Functions

In [2]:
# obtain a table with [slug,category] as column, given retrived get_all url
def get_DApp_table(text):
    DApp_table = []
    DApps_list = html.text.split("}, {")
    for line in DApps_list:
        DApp_line = re.findall(r'"([^"]*)"', line)
        category = DApp_line[DApp_line.index("categories")+1]
        slug = DApp_line[DApp_line.index("slug")+1]
        DApp_table.append([slug,category])
    return DApp_table

In [3]:
# obtain a list with [slug,address] as column, given DApp specific url
def get_DApp_hash(slug):
    DApp_request = 'https://api.stateofthedapps.com/dapps/'
    DApp_html = requests.get(DApp_request + slug)
    DApp_info = re.findall(r'"([^"]*)"', DApp_html.text)
    DApp_hash = DApp_info[DApp_info.index("contractsMainnet")+1:DApp_info.index("contractsRinkeby")]
    DApp_hash = [[hash,slug] for hash in DApp_hash]
    return DApp_hash

Main

In [4]:
# try an arbitarily large page number to get the total number of DApps for ethereum
test_url = 'https://api.stateofthedapps.com/dapps?platform=ethereum&limit=50&page=' + str(23943545)
test = requests.get(test_url)
count = ast.literal_eval(test.text)['pager']['totalCount']
count

1777

In [5]:
# obtain DApp_table with [slug,category] as column
# the first element in DApp_table.shape should match the count from the previous block
base_url = 'https://api.stateofthedapps.com/dapps?platform=ethereum&page='
table = []
for page in range(1,int(np.ceil(count/50)+1)):
    url = base_url + str(page)
    html= requests.get(url)
    table.extend(get_DApp_table(html.text))
DApp_table = pd.DataFrame(table,columns=['slug','category'])
DApp_table.shape

(1777, 2)

In [6]:
# obtain the DApp_hash_table with [slug,address] as column
DApp_hash_table = []
for slug in DApp_table['slug']:
    DApp_hash = get_DApp_hash(slug)
    DApp_hash_table = DApp_hash_table + DApp_hash
DApp_hash_table = pd.DataFrame(DApp_hash_table,columns=['address','slug'])

In [7]:
# merge two tables and save the output 
output = DApp_hash_table.merge(DApp_table,on='slug',how='left')
output.to_csv("DApps.csv")